In [3]:
import pandas as pd
import numpy as np
from fractions import Fraction
from collections import defaultdict      #Generate the value output zero if you didn't mentioned the value to the key in dictionary. 



#Extract the data from excel file.
file_path = 'Data.xlsx'
data = pd.read_excel(file_path)
num_rows, num_cols = data.shape                 #Get Number of Rows and Columns
data_matrix = []                                #Initialize an Empty Matrix:

for i in range(num_rows):
    row_values = data.iloc[i].tolist()
    data_matrix.append(row_values)
data_matrix = pd.DataFrame(data_matrix)

#Shuffled the data, from the original one
data = pd.DataFrame(data_matrix)                                                       #Redundant Code of line, I agree
header = data.iloc[0]                                                                  # Removed the first line of the data and stored in the header variable, keep in the mind in the panda lib they extract the rows and convert it into the column because this is their method.          
data = data[1:]                                                                        #Updated the data variable by removing the first line
shuffled_data = data.sample(frac=1, random_state=42)                                    #Shuffled the data and memorise it by the random state
shuffled_data_with_header = pd.concat([header.to_frame().transpose(), shuffled_data])   #Transformed the header into frame , did transpose and put it into the shuffled data, I didn't mentioned the place so by default it took first position
shuffled_matrix = shuffled_data_with_header.values                                      #Convert back the data to NumbPy 
Training_matrix = shuffled_matrix[0:11]                                                 #Created the Train set
Test_Matrix     = shuffled_matrix[11:15]                                                #Created the Test Matrix


In [4]:
#Prior Probability for the hypothesis. 

matrix_rows, matrix_cols = Training_matrix.shape
specific_column = Training_matrix[1:, matrix_cols - 1]               #Defined the area where I have to look, I removed the first row because it take the "play", and -1 is to clear the dimension.
unique, counts = np.unique(specific_column, return_counts=True)      #Counts the yes or no along with their quantity
unique_counts = dict(zip(unique, counts))                            #Store in the dictionary   
total_samples = len(specific_column)                                 #Total samples of Yes/No
probabilities = {key: Fraction(value, total_samples) for key, value in unique_counts.items()}   #It is giving the prob. but I didn't understand this code
values_probabilities = list(probabilities.values())
Prob_NO = values_probabilities[0]
Prob_YES = values_probabilities[1] 

In [5]:
#Calculate Likhlihood Probabilities of Training Set
def Likelihood_probabilities(Training_matrix, matrix_rows, matrix_cols):
    last_col_index = matrix_cols - 1
    probabilities = []
    
    # Extracting the last column values 
    last_column_values = [row[-1] for row in Training_matrix[1:]]     # In python negative value represents the last value of the matrix or list. 
    
    unique_last_col_values = set(last_column_values)   # Getting unique values in the last column
    
    for col_index in range(matrix_cols - 1):     # Exclude the last column
        col_values = [row[col_index] for row in Training_matrix[1:]]   #  This line extracts the column values for each column. 
        unique_col_values = set(col_values)    # Makeing the set for unique values of column
        
        for value in unique_col_values:
            for last_value in unique_last_col_values:
                total_last_col_value = last_column_values.count(last_value)
                count = sum(1 for row in Training_matrix[1:] if row[col_index] == value and row[-1] == last_value)     # Gives you the count whenever the conditions is true. 
                probability = Fraction(count, total_last_col_value) if total_last_col_value != 0 else 0
                probabilities.append((value, last_value, f"{probability.numerator}/{probability.denominator}" if probability != 0 else 0))
    
    return probabilities


probabilities = Likelihood_probabilities(Training_matrix, matrix_rows, matrix_cols)
print(probabilities)





[('rainy', 'yes', '2/7'), ('rainy', 'no', '1/3'), ('sunny', 'yes', '2/7'), ('sunny', 'no', '2/3'), ('Overcast', 'yes', '3/7'), ('Overcast', 'no', 0), ('mild', 'yes', '3/7'), ('mild', 'no', '2/3'), ('hot', 'yes', '1/7'), ('hot', 'no', '1/3'), ('cool', 'yes', '3/7'), ('cool', 'no', 0), ('high', 'yes', '3/7'), ('high', 'no', '1/1'), ('normal', 'yes', '4/7'), ('normal', 'no', 0), (False, 'yes', '4/7'), (False, 'no', '2/3'), (True, 'yes', '3/7'), (True, 'no', '1/3')]


In [23]:
#Calculate posterior probabilities

def calculate_posterior_individual(Test_Matrix, probabilities, Prob_NO, Prob_YES):
    results = []
    posterior_values = []  # Store posterior probabilities

    for row in Test_Matrix:
        posterior_NO = Prob_NO
        posterior_YES = Prob_YES

        likelihoods = []  # Store likelihood probabilities for this row
        for i, parameter in enumerate(row[:4]):    #Only consider one row at a time
            likelihood = next((prob[2] for prob in probabilities if prob[0] == parameter and prob[1] == 'no'), None)
            if likelihood:
                likelihood = likelihood.split('/')  # Split likelihood probability
                probability = Fraction(int(likelihood[0]), int(likelihood[1]))
                likelihoods.append(probability)

        # Calculate posterior probabilities for this row without accumulating across rows
        for likelihood in likelihoods:
            posterior_NO *= likelihood
            posterior_YES *= (1 - likelihood)

        # Append the individual posterior probabilities for this row
        posterior_values.append((posterior_NO, posterior_YES))
        prediction = "YES" if posterior_YES > posterior_NO else "NO"
        results.append(prediction)
    
    return results, posterior_values

# Calculate posterior probabilities for each row in Test_Matrix
predictions, posterior_probs = calculate_posterior_individual(Test_Matrix, probabilities, Prob_NO, Prob_YES)

for i, (prediction, (posterior_NO, posterior_YES)) in enumerate(zip(predictions, posterior_probs)):
    print(f"For condition of Row {i + 11} of the test_matrix: Play? - {prediction}")
    print(f"Posterior Probability for NO: {posterior_NO}, Posterior Probability for YES: {posterior_YES}")


For condition of Row 11 of the test_matrix: Play? - YES
Posterior Probability for NO: 2/45, Posterior Probability for YES: 7/135
For condition of Row 12 of the test_matrix: Play? - NO
Posterior Probability for NO: 1/45, Posterior Probability for YES: 0
For condition of Row 13 of the test_matrix: Play? - YES
Posterior Probability for NO: 1/15, Posterior Probability for YES: 7/45
For condition of Row 14 of the test_matrix: Play? - YES
Posterior Probability for NO: 1/30, Posterior Probability for YES: 14/45


In [25]:
#Laplace smoothening 
def calculate_posterior_individual_with_smoothing(Test_Matrix, probabilities, Prob_NO, Prob_YES, num_possible_values, smoothing_parameter=1):
    results = []
    posterior_values = []  # Store posterior probabilities

    for row in Test_Matrix:
        posterior_NO = Prob_NO
        posterior_YES = Prob_YES

        likelihoods = []  # Store likelihood probabilities for this row
        for i, parameter in enumerate(row[:4]):    #Only consider one row at a time
            likelihood = next((prob[2] for prob in probabilities if prob[0] == parameter and prob[1] == 'no'), None)
            if likelihood:
                likelihood = likelihood.split('/')  # Split likelihood probability
                probability = Fraction(int(likelihood[0]), int(likelihood[1]))
                likelihoods.append(probability)

        # Laplace smoothing adjustment for posterior probabilities
        for likelihood, v in zip(likelihoods, num_possible_values):
            smoothed_prob = (likelihood.numerator + smoothing_parameter) / (likelihood.denominator + (smoothing_parameter * v))
            posterior_NO *= smoothed_prob
            posterior_YES *= (1 - smoothed_prob)

        # Append the individual posterior probabilities for this row
        posterior_values.append((posterior_NO, posterior_YES))
        prediction = "YES" if posterior_YES > posterior_NO else "NO"
        results.append(prediction)
    
    return results, posterior_values

# Number of possible values for each parameter in Test_Matrix
num_possible_values = [3, 3, 2, 2]

# Calculate posterior probabilities for each row in Test_Matrix with Laplace smoothing
predictions, posterior_probs = calculate_posterior_individual_with_smoothing(Test_Matrix, probabilities, Prob_NO, Prob_YES, num_possible_values)

for i, (prediction, (posterior_NO, posterior_YES)) in enumerate(zip(predictions, posterior_probs)):
    print(f"For condition of Row {i + 11} of the test_matrix: Play? - {prediction}")
    print(f"Posterior Probability for NO: {posterior_NO}, Posterior Probability for YES: {posterior_YES}")


For condition of Row 11 of the test_matrix: Play? - YES
Posterior Probability for NO: 0.029999999999999995, Posterior Probability for YES: 0.09333333333333334
For condition of Row 12 of the test_matrix: Play? - YES
Posterior Probability for NO: 0.01333333333333333, Posterior Probability for YES: 0.046666666666666676
For condition of Row 13 of the test_matrix: Play? - YES
Posterior Probability for NO: 0.049999999999999996, Posterior Probability for YES: 0.23333333333333334
For condition of Row 14 of the test_matrix: Play? - YES
Posterior Probability for NO: 0.033333333333333326, Posterior Probability for YES: 0.31111111111111117


In [36]:
#Think Further
data = pd.read_csv('Flower_Data.csv', header=None)

# Calculate the average of each column (excluding the last column)
averages = data.iloc[:, :-1].mean()    #By default it calculates the mean value of all column excludes the last one. 

# Create a matrix where values > average are set to 1 and others to 0
comparison_matrix = (data.iloc[:, :-1] > averages).astype(int)

# Get the last column of the original data and attach it to the comparison matrix
last_column = data.iloc[:, -1]
comparison_matrix['Last Column'] = last_column

# Save the comparison matrix (including the last column) to a CSV file
comparison_matrix.to_csv('comparison_matrix_with_last_column.csv', index=False)
